# Kontuś

In [1]:
from parse import parse_task_1_2

In [2]:
meta, sentences = parse_task_1_2('training/file_39.ttl')

In [3]:
sent = list(sentences.keys())[0]

In [4]:
sent

'Knowledge Services and Dotstaff is incredibly happy to be a part of this wonderful community, said Joe Bielawski, the president of Knowledge Services.'

# Cezik i Rzegi

In [5]:
from entities_recognition_tester import process_using_spacy
import en_core_web_sm

In [6]:
pary = process_using_spacy(sent, en_core = en_core_web_sm.load(), format_result=True)
pary

[('Dotstaff', 'ORG'),
 ('Joe Bielawski', 'PERSON'),
 ('Knowledge Services', 'ORG')]

In [7]:
import itertools
perm = list(itertools.permutations(pary, 2))
perm

[(('Dotstaff', 'ORG'), ('Joe Bielawski', 'PERSON')),
 (('Dotstaff', 'ORG'), ('Knowledge Services', 'ORG')),
 (('Joe Bielawski', 'PERSON'), ('Dotstaff', 'ORG')),
 (('Joe Bielawski', 'PERSON'), ('Knowledge Services', 'ORG')),
 (('Knowledge Services', 'ORG'), ('Dotstaff', 'ORG')),
 (('Knowledge Services', 'ORG'), ('Joe Bielawski', 'PERSON'))]

In [8]:
big_dict = {
     ('Organisation', 'Organisation'): ['affiliation'],
     ('Person', 'EducationalInstitution'): ['almaMater'],
     ('Band', 'Person'): ['formerBandMember', 'bandMember'],
     ('Person', 'Place'): ['deathPlace', 'birthPlace'],
     ('Organisation', 'Person'): ['president', 'ceo'],
     ('Person', 'Person'): ['spouse', 'relative', 'parent', 'child'],
     ('Athlete', 'SportsTeam'): ['formerTeam', 'debutTeam', 'club'],
     ('Organisation', 'Country'): ['country'],
     ('Person', 'Country'): ['country'],
     ('Place', 'Country'): ['country'],
     ('PopulatedPlace', 'PopulatedPlace'): ['department'],
     ('Place', 'PopulatedPlace'): ['district'],
     ('Scientist', 'Person'): ['doctoralStudent', 'doctoralAdvisor'],
     ('Person', 'Organisation'): ['employer'],
     ('Organisation', 'City'): ['foundationPlace'],
     ('Organisation', 'PopulatedPlace'): ['headquarter'],
     ('Organisation', 'Settlement'): ['hometown'],
     ('Person', 'Settlement'): ['hometown'],
     ('PopulatedPlace', 'Person'): ['leaderName'],
     ('Place', 'Place'): ['locatedInArea'],
     ('Organisation_Person_Place', 'Place'): ['location'],
     ('Person', 'Country'): ['nationality'],
     ('Company', 'Company'): ['subsidiary'],
     ('ArchitecturalStructure', 'Organisation'): ['tenant'],
     ('Athlete', 'Person'): ['trainer']
}

In [9]:
maps = {
    'ORG': 'Organisation',
    'PERSON': 'Person'
}

In [10]:
for a, b in perm:
    print(big_dict[(maps[a[1]], maps[b[1]])])

['president', 'ceo']
['affiliation']
['employer']
['employer']
['affiliation']
['president', 'ceo']


# Marian i Ptok

In [11]:
from sparql_resource_property_getter import get_resource, find_property

In [18]:
for thing, typ in pary:
    print(get_resource(thing))

[]
[]
[]
